In [ ]:
#mount google drive
from google.colab import drive
drive.mount('drive', force_remount=True)

Mounted at drive


In [ ]:
#convert mp4 to wav

import subprocess

def convert_m4a_to_wav(input_path, output_path):
    command = ['ffmpeg', '-i', input_path, output_path]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Replace these paths with your actual input and output paths
input_m4a_path = 'wavs/m1.m4a'
output_wav_path = 'wavs2/m1.wav'

convert_m4a_to_wav(input_m4a_path, output_wav_path)


In [ ]:
# move files from subdirectories to /wavs

import os
import shutil
def move_files(src_dir, dest_dir):
    count=0
    for foldername, subfolders, filenames in os.walk(src_dir):
        for filename in filenames:
            count+=1
            src_path = os.path.join(foldername, filename)
            dest_path = os.path.join(dest_dir, filename)
            shutil.move(src_path, dest_path)
    print(count)
# delete empty directories
def delete_empty_dirs(dir_path):
    for foldername, subfolders, filenames in os.walk(dir_path, topdown=False):
        for subfolder in subfolders:
            folder_path = os.path.join(foldername, subfolder)
            if not os.listdir(folder_path):
                print(foldername)
                os.rmdir(folder_path)



src_directory = '/content/drive/MyDrive/x/wavs'
dest_directory = '/content/drive/MyDrive/x/wavs'

move_files(src_directory, dest_directory)
delete_empty_dirs(src_directory)


31081


In [ ]:
#change stereo to mono

stereo=False
if stereo:
    !pip install pydub
    from pydub import AudioSegment

    # Define the directory containing your audio files
    audio_directory = '/content/drive/MyDrive/x/wavs'

    # Get a list of all audio files in the directory
    audio_files = [file for file in os.listdir(audio_directory) if file.endswith(".wav")]

    # Iterate through the audio files
    for audio_file in audio_files:
        # Construct the full file path
        input_audio_path = os.path.join(audio_directory, audio_file)

        # Load the audio
        audio = AudioSegment.from_wav(input_audio_path)

        # Check if the audio is stereo
        if audio.channels >= 2:
            # Convert stereo to mono
            audio2 = audio.set_channels(1)

            # Construct the output file path (you can change the output directory if needed)
            output_audio_path = os.path.join(audio_directory, audio_file)

            # Export the mono audio to a new file
            audio2.export(output_audio_path, format="wav")


            print(f"Converted {audio_file} to mono.")

    print("Stereo to mono conversion for all files complete.")


In [ ]:
#rename files to standardise them (4_speakers database)
import os
def rename(folder_name):
  source_directory = f"/content/drive/MyDrive/{folder_name}/"

  # List all files in the source directory
  file_list = os.listdir(source_directory)

  for old_name in file_list:
    if old_name.endswith(".wav") and len(old_name)>8:
        new_name=old_name[-8:]  # retain 1234.wav
        old_path = os.path.join(source_directory, old_name)
        new_path = os.path.join(source_directory, new_name)
        # Rename the file
        os.rename(old_path, new_path)
        print(f"Renamed: {old_name} -> {new_name}")
rename('4_speakers/x/wavs')

In [ ]:
#delete any lines in list.txt if there is no matching audio pair
def delete_lines(transcript):
  with open(transcript, "r") as f:
    lines = f.readlines()

  adjusted_lines = []
  for line in lines:
      path_to_audio = '/content/tacotron2/'+ line.split('|')[0][:-4] + '.wav'
      if os.path.exists(path_to_audio):
          adjusted_lines.append(line)
      if not os.path.exists(path_to_audio):
          print(path_to_audio)

  with open(transcript, "w") as f:
    f.writelines(adjusted_lines)

delete_lines("/content/tacotron2/filelists/list.txt")


In [ ]:
#split files into train and validate

with open('/content/tacotron2/filelists/list.txt', 'r') as list_file:
  wav_files = [line.strip() for line in list_file]

random.shuffle(wav_files)
train_to=round(len(wav_files)*.8)


# Create train_files.txt
train_files = wav_files[:train_to]
train_files.sort()
with open(os.path.join('filelists', 'train_files.txt'), 'w') as f_train:
    for filename in train_files:
        f_train.write(os.path.join(filename) + '\n')

# Create val_files.txt
val_files = wav_files[train_to:]
val_files.sort()
with open(os.path.join('filelists', 'val_files.txt'), 'w') as f_val:
    for filename in val_files:
        f_val.write(os.path.join(filename) + '\n')


In [ ]:
#fix the 4_speakers database transcription format
input_file = "/content/drive/MyDrive/4_speakers/metadata.txt"
output_file = "/content/drive/MyDrive/4_speakers/list.txt"

with open(input_file, "r") as f:
    lines = f.readlines()

modified_lines = []

for line in lines:
    characters_to_remove = 16
    number=line[10:14]
    line = line.replace(line[:16],number)

    line=line[:-5]

    modified_lines.append(line + "\n")

with open(output_file, "w") as f:
    f.writelines(modified_lines)

print("File modified and saved as", output_file)


File modified and saved as /content/drive/MyDrive/4_speakers/listt.txt


In [ ]:
#fix VCTK file names

if choosedata=='vctk':
    folder_path = 'wavs'
    files = os.listdir(folder_path)

    for filename in files:
        if filename.endswith("_mic1.wav"):
            new_filename = filename.replace("_mic1.wav", ".wav")
            old_path = os.path.join(folder_path, filename)
            new_path = os.path.join(folder_path, new_filename)
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} to {new_filename}")


In [ ]:
#change audio formtat (bits per sample and sample rate), also only take a percentage of files

import subprocess
!apt-get -qq install sox
import wave
import os
import random
random.seed(0)


def convert_audio(input_file_path, output_file_path):
    !sox -v 0.95 "$input_file_path" -r 22050 -b 16 "$output_file_path"

def process_audio_files(input_dir, output_dir):

    audio_files = os.listdir(input_dir)
    audio_files.sort()
    selected_files=audio_files
    percent=1
    split_point=round(len(audio_files)*percent)
    #random.shuffle(selected_files)
    selected_files=selected_files[:308]
    selected_files.sort()
    amo=len(selected_files)

    for index, file_name in enumerate(selected_files):
        input_file_path = os.path.join(input_dir, file_name)
        output_file_path = os.path.join(output_dir, file_name)
        convert_audio(input_file_path, output_file_path)
        print(f"file {index} of {amo} format changed")


input_directory = '/content/drive/MyDrive/4_speakers/sam/wavs'
output_directory = '/content/drive/MyDrive/4_speakers/sam2/wavs'

if not os.path.exists(output_directory):
      os.makedirs(output_directory)

process_audio_files(input_directory, output_directory)


In [ ]:
#TRIM OUT SILENCE from beginning and end of clips

!pip install pydub
from pydub import AudioSegment
from pydub.silence import detect_leading_silence, detect_leading_silence

def trim_start_and_end(audio_path, output_path, silence_threshold,silence_threshold2):
    # Load the audio clip
    audio = AudioSegment.from_file(audio_path, format="wav")

    # Find the start and end of non-silent sections
    start_trim = detect_leading_silence(audio, silence_threshold)
    end_trim = detect_leading_silence(audio.reverse(), silence_threshold2)

    # Calculate the new audio with trimmed start and end
    trimmed_audio = audio[start_trim:len(audio)-end_trim]

    # Export the trimmed audio to the specified output path
    trimmed_audio.export(output_path, format="wav")

input_folder = '/content/drive/MyDrive/wavs'
output_folder = '/content/drive/MyDrive/wavs2'

if not os.path.exists(output_folder):
        os.makedirs(output_folder)

for filename in os.listdir(input_folder):
        input_audio_path = os.path.join(input_folder, filename)
        output_audio_path = os.path.join(output_folder, filename)
        trim_start_and_end(input_audio_path, output_audio_path, -40,-50)    #increasing (bringing closer to 0) cuts more audio

print("Trimming complete.")

Trimming complete.
